In [1]:
import pandas as pd
import numpy as np
import math
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, const, LpStatus, value

# the seed number should be your student ID. 
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(1234) 

# Shipment cost from factory to distribution center
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 12),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (4,3))
CostFtoD = pd.DataFrame(RandomFtoD2, columns=['Atlas', 'Nebule',"Azure"], 
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoD # gives the shipment cost from factory to distribution center

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (3,5))
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"], 
                        index=['Atlas', 'Nebule',"Azure"])
#CostDtoS # gives the shipment cost from distribution center to store

# Shipment cost from factory to store
RandomFtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 20),0).astype(int)
RandomFtoS2 = np.reshape(RandomFtoS1, (4,5))
CostFtoS = pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoS # gives the shipment cost from distribution center to store

# Your Required Information About Shipment Cost
print("The shipment cost from factory to distribution center")
print(CostFtoD)
print("\n The shipment cost from distribution center to store")
print(CostDtoS)
print("\n The shipment cost from factory to store")
print(CostFtoS)

The shipment cost from factory to distribution center
             Atlas  Nebule  Azure
Tonoz            4       9      3
White Kiosk      2       8      3
Quarterage       2       3      4
Middle Yard      1       9      6

 The shipment cost from distribution center to store
        Venus  Minerva  Neptunus  Mars  Ceres
Atlas       4        5         4     8      2
Nebule      1        1         6     7      5
Azure       2        8         9    10      9

 The shipment cost from factory to store
             Venus  Minerva  Neptunus  Mars  Ceres
Tonoz            5        5         3     8      7
White Kiosk      5        2         9     4      9
Quarterage       5        8         8     3      4
Middle Yard      1        9         8     6      5


In [2]:
CostDtoS

,Venus,Minerva,Neptunus,Mars,Ceres
Atlas,4,5,4,8,2
Nebule,1,1,6,7,5
Azure,2,8,9,10,9


In [3]:
CostFtoS

,Venus,Minerva,Neptunus,Mars,Ceres
Tonoz,5,5,3,8,7
White Kiosk,5,2,9,4,9
Quarterage,5,8,8,3,4
Middle Yard,1,9,8,6,5


In [4]:

whCaps = np.array([3, 5, 10, 7]) * 1000
WareHouseCapacities = pd.DataFrame(whCaps.T, index=["Tonoz,White Kiosk,Quarterage,Middle Yard".split(",")], columns=["Capacities"])
WareHouseCapacities

,Capacities
Tonoz,3000
White Kiosk,5000
Quarterage,10000
Middle Yard,7000


In [5]:
storeDemands = pd.DataFrame(np.array([5500, 4750, 6550, 4000, 4200]).T, index=["Venus Minerva Neptunus Mars Ceres".split(" ")], columns=["Demands"])
storeDemands

,Demands
Venus,5500
Minerva,4750
Neptunus,6550
Mars,4000
Ceres,4200


In [6]:
CostFtoD # Factory to Distribution Center
CostDtoS # Distribution to Store
CostFtoS # Factory to Store

,Venus,Minerva,Neptunus,Mars,Ceres
Tonoz,5,5,3,8,7
White Kiosk,5,2,9,4,9
Quarterage,5,8,8,3,4
Middle Yard,1,9,8,6,5


In [7]:
WareHouses = list(CostFtoS.index)
Distributors = list(CostDtoS.index)
Stores = list(CostDtoS.columns)

# Model Construction

In [8]:
problem = LpProblem("Assignment2-Distribution-Problem", LpMinimize)

In [9]:
X = LpVariable.dicts("x",
                     (WareHouses, Distributors),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Y = LpVariable.dicts("x",
                     (Distributors, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Z = LpVariable.dicts("x",
                     (WareHouses, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

In [10]:
# Adding variables / since we use pd Dataframe, the first index must be the col
# Thus, this makes the indexes go opposite sides
# Warehouse and Distribution is the SAME THING

problem_F_to_D = lpSum(CostFtoD[distributor][wareHouse] * X[wareHouse][distributor] \
    for distributor in Distributors for wareHouse in WareHouses)

problem_D_to_S = lpSum(CostDtoS[store][distributor] * Y[distributor][store] \
                    for distributor in Distributors for store in Stores)

problem_F_to_S = lpSum(CostFtoS[store][wareHouse] * Z[wareHouse][store] \
    for wareHouse in WareHouses for store in Stores)
# objective function
problem +=  problem_F_to_D + problem_D_to_S + problem_F_to_S

In [11]:
# checking the model translation condition
print(str(problem_F_to_D))
print(CostFtoD)


x_Middle_Yard_Atlas + 6*x_Middle_Yard_Azure + 9*x_Middle_Yard_Nebule + 2*x_Quarterage_Atlas + 4*x_Quarterage_Azure + 3*x_Quarterage_Nebule + 4*x_Tonoz_Atlas + 3*x_Tonoz_Azure + 9*x_Tonoz_Nebule + 2*x_White_Kiosk_Atlas + 3*x_White_Kiosk_Azure + 8*x_White_Kiosk_Nebule
             Atlas  Nebule  Azure
Tonoz            4       9      3
White Kiosk      2       8      3
Quarterage       2       3      4
Middle Yard      1       9      6


#### They seem correct, the defined model and the pulp values are the same 

# Constraints

In [12]:
for store in Stores:
    # Factory -> Store + WareHouse -> Store >= Demands
    problem += (lpSum(Y[distributor][store] for distributor in Distributors) + lpSum(Z[wareHouse][store] for wareHouse in WareHouses)) >= storeDemands["Demands"][store]

In [13]:
for wareHouse in WareHouses:
    # WareHouse -> DistributionCenter + WareHouse -> Store <= Capacities
    problem += (lpSum(X[wareHouse][distributor] for distributor in Distributors) + lpSum(Z[wareHouse][store] for store in Stores)) <= WareHouseCapacities["Capacities"][wareHouse]

In [14]:
for distributor in Distributors:
    # for what the distributors take from the Warehouses, what it gives to the stores
    # must be equal or greater so that they can provide the resources
    problem += lpSum(X[wareHouse][distributor] for wareHouse in WareHouses) >= lpSum(Y[distributor][store] for store in Stores)

In [15]:
problem.solve()

1

In [16]:
print("The problem is ", LpStatus[problem.status])
print("The objective function value is ", value(problem.objective))
for v in problem.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)

The problem is  Optimal
The objective function value is  72600.0
x_Atlas_Neptunus = 3550.0
x_Middle_Yard_Atlas = 1500.0
x_Middle_Yard_Venus = 5500.0
x_Quarterage_Atlas = 1800.0
x_Quarterage_Ceres = 4200.0
x_Quarterage_Mars = 4000.0
x_Tonoz_Neptunus = 3000.0
x_White_Kiosk_Atlas = 250.0
x_White_Kiosk_Minerva = 4750.0


In [17]:
print(problem.variables())

[x_Atlas_Ceres, x_Atlas_Mars, x_Atlas_Minerva, x_Atlas_Neptunus, x_Atlas_Venus, x_Azure_Ceres, x_Azure_Mars, x_Azure_Minerva, x_Azure_Neptunus, x_Azure_Venus, x_Middle_Yard_Atlas, x_Middle_Yard_Azure, x_Middle_Yard_Ceres, x_Middle_Yard_Mars, x_Middle_Yard_Minerva, x_Middle_Yard_Nebule, x_Middle_Yard_Neptunus, x_Middle_Yard_Venus, x_Nebule_Ceres, x_Nebule_Mars, x_Nebule_Minerva, x_Nebule_Neptunus, x_Nebule_Venus, x_Quarterage_Atlas, x_Quarterage_Azure, x_Quarterage_Ceres, x_Quarterage_Mars, x_Quarterage_Minerva, x_Quarterage_Nebule, x_Quarterage_Neptunus, x_Quarterage_Venus, x_Tonoz_Atlas, x_Tonoz_Azure, x_Tonoz_Ceres, x_Tonoz_Mars, x_Tonoz_Minerva, x_Tonoz_Nebule, x_Tonoz_Neptunus, x_Tonoz_Venus, x_White_Kiosk_Atlas, x_White_Kiosk_Azure, x_White_Kiosk_Ceres, x_White_Kiosk_Mars, x_White_Kiosk_Minerva, x_White_Kiosk_Nebule, x_White_Kiosk_Neptunus, x_White_Kiosk_Venus]


In [18]:
print(problem)

Assignment2-Distribution-Problem:
MINIMIZE
2*x_Atlas_Ceres + 8*x_Atlas_Mars + 5*x_Atlas_Minerva + 4*x_Atlas_Neptunus + 4*x_Atlas_Venus + 9*x_Azure_Ceres + 10*x_Azure_Mars + 8*x_Azure_Minerva + 9*x_Azure_Neptunus + 2*x_Azure_Venus + 1*x_Middle_Yard_Atlas + 6*x_Middle_Yard_Azure + 5*x_Middle_Yard_Ceres + 6*x_Middle_Yard_Mars + 9*x_Middle_Yard_Minerva + 9*x_Middle_Yard_Nebule + 8*x_Middle_Yard_Neptunus + 1*x_Middle_Yard_Venus + 5*x_Nebule_Ceres + 7*x_Nebule_Mars + 1*x_Nebule_Minerva + 6*x_Nebule_Neptunus + 1*x_Nebule_Venus + 2*x_Quarterage_Atlas + 4*x_Quarterage_Azure + 4*x_Quarterage_Ceres + 3*x_Quarterage_Mars + 8*x_Quarterage_Minerva + 3*x_Quarterage_Nebule + 8*x_Quarterage_Neptunus + 5*x_Quarterage_Venus + 4*x_Tonoz_Atlas + 3*x_Tonoz_Azure + 7*x_Tonoz_Ceres + 8*x_Tonoz_Mars + 5*x_Tonoz_Minerva + 9*x_Tonoz_Nebule + 3*x_Tonoz_Neptunus + 5*x_Tonoz_Venus + 2*x_White_Kiosk_Atlas + 3*x_White_Kiosk_Azure + 9*x_White_Kiosk_Ceres + 4*x_White_Kiosk_Mars + 2*x_White_Kiosk_Minerva + 8*x_White_Kio

# Final Results

### B) What is the value of objective function?

In [19]:
result_of_b = value(problem.objective)
print("The value of the objective function: ", result_of_b)

The value of the objective function:  72600.0


### c) What is the value of decision variables? According to this values, evaluate the supply chain and flow in this supply chain?

In [22]:
print("\n The NON-ZERO VARIABLE values are: ")
print(*[f"{v.name}: {v.varValue}"  for v in problem.variables() if v.varValue != 0], sep="\n")
print("\n THE ZERO VARIABLE values are: ")
print(*[f"{v.name}: {v.varValue}"  for v in problem.variables() if v.varValue == 0], sep="\n")


 The NON-ZERO VARIABLE values are: 
x_Atlas_Neptunus: 3550.0
x_Middle_Yard_Atlas: 1500.0
x_Middle_Yard_Venus: 5500.0
x_Quarterage_Atlas: 1800.0
x_Quarterage_Ceres: 4200.0
x_Quarterage_Mars: 4000.0
x_Tonoz_Neptunus: 3000.0
x_White_Kiosk_Atlas: 250.0
x_White_Kiosk_Minerva: 4750.0

 THE ZERO VARIABLE values are: 
x_Atlas_Ceres: 0.0
x_Atlas_Mars: 0.0
x_Atlas_Minerva: 0.0
x_Atlas_Venus: 0.0
x_Azure_Ceres: 0.0
x_Azure_Mars: 0.0
x_Azure_Minerva: 0.0
x_Azure_Neptunus: 0.0
x_Azure_Venus: 0.0
x_Middle_Yard_Azure: 0.0
x_Middle_Yard_Ceres: 0.0
x_Middle_Yard_Mars: 0.0
x_Middle_Yard_Minerva: 0.0
x_Middle_Yard_Nebule: 0.0
x_Middle_Yard_Neptunus: 0.0
x_Nebule_Ceres: 0.0
x_Nebule_Mars: 0.0
x_Nebule_Minerva: 0.0
x_Nebule_Neptunus: 0.0
x_Nebule_Venus: 0.0
x_Quarterage_Azure: 0.0
x_Quarterage_Minerva: 0.0
x_Quarterage_Nebule: 0.0
x_Quarterage_Neptunus: 0.0
x_Quarterage_Venus: 0.0
x_Tonoz_Atlas: 0.0
x_Tonoz_Azure: 0.0
x_Tonoz_Ceres: 0.0
x_Tonoz_Mars: 0.0
x_Tonoz_Minerva: 0.0
x_Tonoz_Nebule: 0.0
x_Tonoz_Ve

### d) If we change the capacity of warehouse Quarterage from 10000 to 7500, and the demand of Ceres from 4200 to 1700, how do the objective value and the values of the decision variables change?

In [26]:
import pandas as pd
import numpy as np
import math
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, const, LpStatus, value

# the seed number should be your student ID. 
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(1234) 

# Shipment cost from factory to distribution center
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 12),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (4,3))
CostFtoD = pd.DataFrame(RandomFtoD2, columns=['Atlas', 'Nebule',"Azure"], 
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoD # gives the shipment cost from factory to distribution center

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (3,5))
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"], 
                        index=['Atlas', 'Nebule',"Azure"])
#CostDtoS # gives the shipment cost from distribution center to store

# Shipment cost from factory to store
RandomFtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 20),0).astype(int)
RandomFtoS2 = np.reshape(RandomFtoS1, (4,5))
CostFtoS = pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoS # gives the shipment cost from distribution center to store

# Your Required Information About Shipment Cost
whCaps = np.array([3, 5, 7.5, 7]) * 1000
WareHouseCapacities = pd.DataFrame(whCaps.T, index=["Tonoz,White Kiosk,Quarterage,Middle Yard".split(",")], columns=["Capacities"])
storeDemands = pd.DataFrame(np.array([5500, 4750, 6550, 4000, 1700]).T, index=["Venus Minerva Neptunus Mars Ceres".split(" ")], columns=["Demands"])

WareHouses = list(CostFtoS.index)
Distributors = list(CostDtoS.index)
Stores = list(CostDtoS.columns)


In [27]:

problem_D = LpProblem("Assignment2-Distribution-Problem", LpMinimize)

X = LpVariable.dicts("x",
                     (WareHouses, Distributors),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Y = LpVariable.dicts("x",
                     (Distributors, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Z = LpVariable.dicts("x",
                     (WareHouses, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

# Adding variables / since we use pd Dataframe, the first index must be the col
# Thus, this makes the indexes go opposite sides
# Warehouse and Distribution is the SAME THING

problem_F_to_D = lpSum(CostFtoD[distributor][wareHouse] * X[wareHouse][distributor] \
    for distributor in Distributors for wareHouse in WareHouses)

problem_D_to_S = lpSum(CostDtoS[store][distributor] * Y[distributor][store] \
                    for distributor in Distributors for store in Stores)

problem_F_to_S = lpSum(CostFtoS[store][wareHouse] * Z[wareHouse][store] \
    for wareHouse in WareHouses for store in Stores)
# objective function
problem_D +=  problem_F_to_D + problem_D_to_S + problem_F_to_S

for store in Stores:
    # Factory -> Store + WareHouse -> Store >= Demands
    problem_D += (lpSum(Y[distributor][store] for distributor in Distributors) + lpSum(Z[wareHouse][store] for wareHouse in WareHouses)) >= storeDemands["Demands"][store]

for wareHouse in WareHouses:
    # WareHouse -> DistributionCenter + WareHouse -> Store <= Capacities
    problem_D += (lpSum(X[wareHouse][distributor] for distributor in Distributors) + lpSum(Z[wareHouse][store] for store in Stores)) <= WareHouseCapacities["Capacities"][wareHouse]

for distributor in Distributors:
    # for what the distributors take from the Warehouses, what it gives to the stores
    # must be equal or greater so that they can provide the resources
    problem_D += lpSum(X[wareHouse][distributor] for wareHouse in WareHouses) >= lpSum(Y[distributor][store] for store in Stores)
    
problem_D.solve()


1

In [28]:

print("The problem is ", LpStatus[problem_D.status])
print("The objective function value is ", value(problem_D.objective))
for v in problem_D.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)

The problem is  Optimal
The objective function value is  62600.0
x_Atlas_Neptunus = 3550.0
x_Middle_Yard_Atlas = 1500.0
x_Middle_Yard_Venus = 5500.0
x_Quarterage_Atlas = 1800.0
x_Quarterage_Ceres = 1700.0
x_Quarterage_Mars = 4000.0
x_Tonoz_Neptunus = 3000.0
x_White_Kiosk_Atlas = 250.0
x_White_Kiosk_Minerva = 4750.0


### e) If we change the shipment cost from warehouse Tonoz to distribution center Atlas as 10 and Azure to Minerva as 5, how the objective function is affected. Give new objective value and number of shipments, and then explain your answer with few sentences.

In [35]:
import pandas as pd
import numpy as np
import math
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, const, LpStatus, value

# the seed number should be your student ID. 
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(1234) 

# Shipment cost from factory to distribution center
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 12),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (4,3))
CostFtoD = pd.DataFrame(RandomFtoD2, columns=['Atlas', 'Nebule',"Azure"], 
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoD # gives the shipment cost from factory to distribution center
CostFtoD["Atlas"]["Tonoz"] = 10
print(CostFtoD)
# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (3,5))
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"], 
                        index=['Atlas', 'Nebule',"Azure"])
#CostDtoS # gives the shipment cost from distribution center to store
CostDtoS["Minerva"]["Azure"] = 5
print(CostDtoS)
# Shipment cost from factory to store
RandomFtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 20),0).astype(int)
RandomFtoS2 = np.reshape(RandomFtoS1, (4,5))
CostFtoS = pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoS # gives the shipment cost from distribution center to store

# Your Required Information About Shipment Cost
whCaps = np.array([3, 5, 10, 7]) * 1000
WareHouseCapacities = pd.DataFrame(whCaps.T, index=["Tonoz,White Kiosk,Quarterage,Middle Yard".split(",")], columns=["Capacities"])
storeDemands = pd.DataFrame(np.array([5500, 4750, 6550, 4000, 4200]).T, index=["Venus Minerva Neptunus Mars Ceres".split(" ")], columns=["Demands"])

WareHouses = list(CostFtoS.index)
Distributors = list(CostDtoS.index)
Stores = list(CostDtoS.columns)


             Atlas  Nebule  Azure
Tonoz           10       9      3
White Kiosk      2       8      3
Quarterage       2       3      4
Middle Yard      1       9      6
        Venus  Minerva  Neptunus  Mars  Ceres
Atlas       4        5         4     8      2
Nebule      1        1         6     7      5
Azure       2        5         9    10      9


In [36]:

problem_E = LpProblem("Assignment2-Distribution-Problem", LpMinimize)

X = LpVariable.dicts("x",
                     (WareHouses, Distributors),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Y = LpVariable.dicts("x",
                     (Distributors, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Z = LpVariable.dicts("x",
                     (WareHouses, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

# Adding variables / since we use pd Dataframe, the first index must be the col
# Thus, this makes the indexes go opposite sides
# Warehouse and Distribution is the SAME THING

problem_F_to_D = lpSum(CostFtoD[distributor][wareHouse] * X[wareHouse][distributor] \
    for distributor in Distributors for wareHouse in WareHouses)

problem_D_to_S = lpSum(CostDtoS[store][distributor] * Y[distributor][store] \
                    for distributor in Distributors for store in Stores)

problem_F_to_S = lpSum(CostFtoS[store][wareHouse] * Z[wareHouse][store] \
    for wareHouse in WareHouses for store in Stores)
# objective function
problem_E +=  problem_F_to_D + problem_D_to_S + problem_F_to_S

for store in Stores:
    # Factory -> Store + WareHouse -> Store >= Demands
    problem_E += (lpSum(Y[distributor][store] for distributor in Distributors) + lpSum(Z[wareHouse][store] for wareHouse in WareHouses)) >= storeDemands["Demands"][store]

for wareHouse in WareHouses:
    # WareHouse -> DistributionCenter + WareHouse -> Store <= Capacities
    problem_E += (lpSum(X[wareHouse][distributor] for distributor in Distributors) + lpSum(Z[wareHouse][store] for store in Stores)) <= WareHouseCapacities["Capacities"][wareHouse]

for distributor in Distributors:
    # for what the distributors take from the Warehouses, what it gives to the stores
    # must be equal or greater so that they can provide the resources
    problem_E += lpSum(X[wareHouse][distributor] for wareHouse in WareHouses) >= lpSum(Y[distributor][store] for store in Stores)
    
problem_E.solve()


1

In [37]:
print()
print("The problem is ", LpStatus[problem_E.status])
print("The objective function value is ", value(problem_E.objective))
print()
for v in problem_E.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)


The problem is  Optimal
The objective function value is  72600.0

x_Atlas_Neptunus = 3550.0
x_Middle_Yard_Atlas = 1500.0
x_Middle_Yard_Venus = 5500.0
x_Quarterage_Atlas = 1800.0
x_Quarterage_Ceres = 4200.0
x_Quarterage_Mars = 4000.0
x_Tonoz_Neptunus = 3000.0
x_White_Kiosk_Atlas = 250.0
x_White_Kiosk_Minerva = 4750.0


### f)  we change the capacity of the warehouse Middle Yard from 7000 to 5000, what should we expect? Can we obtain optimal solution? Please explain your conclusion clearly. If it is necessary make some arrangement and give the results with explanations.

In [38]:
import pandas as pd
import numpy as np
import math
from pulp import LpVariable, LpProblem, LpMinimize, lpSum, const, LpStatus, value

# the seed number should be your student ID. 
# If it has letter, please ignore it and use only numbers in the ID
np.random.seed(1234) 

# Shipment cost from factory to distribution center
RandomFtoD1 = np.round(np.random.uniform(low = 1, high = 10, size = 12),0).astype(int)
RandomFtoD2 = np.reshape(RandomFtoD1, (4,3))
CostFtoD = pd.DataFrame(RandomFtoD2, columns=['Atlas', 'Nebule',"Azure"], 
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoD # gives the shipment cost from factory to distribution center

# Shipment cost from distribution center to store
RandomDtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 15),0).astype(int)
RandomDtoS2 = np.reshape(RandomDtoS1, (3,5))
CostDtoS = pd.DataFrame(RandomDtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"], 
                        index=['Atlas', 'Nebule',"Azure"])
#CostDtoS # gives the shipment cost from distribution center to store

# Shipment cost from factory to store
RandomFtoS1 = np.round(np.random.uniform(low = 1, high = 10, size = 20),0).astype(int)
RandomFtoS2 = np.reshape(RandomFtoS1, (4,5))
CostFtoS = pd.DataFrame(RandomFtoS2, columns=["Venus", "Minerva", "Neptunus", "Mars", "Ceres"],
                        index=["Tonoz","White Kiosk","Quarterage","Middle Yard"])
#CostFtoS # gives the shipment cost from distribution center to store

# Your Required Information About Shipment Cost
whCaps = np.array([3, 5, 10, 5]) * 1000
WareHouseCapacities = pd.DataFrame(whCaps.T, index=["Tonoz,White Kiosk,Quarterage,Middle Yard".split(",")], columns=["Capacities"])
storeDemands = pd.DataFrame(np.array([5500, 4750, 6550, 4000, 4200]).T, index=["Venus Minerva Neptunus Mars Ceres".split(" ")], columns=["Demands"])

WareHouses = list(CostFtoS.index)
Distributors = list(CostDtoS.index)
Stores = list(CostDtoS.columns)


In [39]:

problem_F = LpProblem("Assignment2-Distribution-Problem", LpMinimize)

X = LpVariable.dicts("x",
                     (WareHouses, Distributors),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Y = LpVariable.dicts("x",
                     (Distributors, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

Z = LpVariable.dicts("x",
                     (WareHouses, Stores),
                     lowBound = 0,
                     upBound = None,
                     cat = const.LpInteger)

# Adding variables / since we use pd Dataframe, the first index must be the col
# Thus, this makes the indexes go opposite sides
# Warehouse and Distribution is the SAME THING

problem_F_to_D = lpSum(CostFtoD[distributor][wareHouse] * X[wareHouse][distributor] \
    for distributor in Distributors for wareHouse in WareHouses)

problem_D_to_S = lpSum(CostDtoS[store][distributor] * Y[distributor][store] \
                    for distributor in Distributors for store in Stores)

problem_F_to_S = lpSum(CostFtoS[store][wareHouse] * Z[wareHouse][store] \
    for wareHouse in WareHouses for store in Stores)
# objective function
problem_F +=  problem_F_to_D + problem_D_to_S + problem_F_to_S

for store in Stores:
    # Factory -> Store + WareHouse -> Store >= Demands
    problem_F += (lpSum(Y[distributor][store] for distributor in Distributors) + lpSum(Z[wareHouse][store] for wareHouse in WareHouses)) >= storeDemands["Demands"][store]

for wareHouse in WareHouses:
    # WareHouse -> DistributionCenter + WareHouse -> Store <= Capacities
    problem_F += (lpSum(X[wareHouse][distributor] for distributor in Distributors) + lpSum(Z[wareHouse][store] for store in Stores)) <= WareHouseCapacities["Capacities"][wareHouse]

for distributor in Distributors:
    # for what the distributors take from the Warehouses, what it gives to the stores
    # must be equal or greater so that they can provide the resources
    problem_F += lpSum(X[wareHouse][distributor] for wareHouse in WareHouses) >= lpSum(Y[distributor][store] for store in Stores)
    
problem_F.solve()


-1

In [40]:

print("The problem is ", LpStatus[problem_F.status])
print("The objective function value is ", value(problem_F.objective))
print()
for v in problem_F.variables():
    if v.varValue > 0.1:
        print(v.name, "=", v.varValue)

The problem is  Infeasible
The objective function value is  74600.0

x_Atlas_Neptunus = 3550.0
x_Azure_Venus = 2000.0
x_Middle_Yard_Atlas = 1500.0
x_Middle_Yard_Venus = 3500.0
x_Quarterage_Atlas = 1800.0
x_Quarterage_Ceres = 4200.0
x_Quarterage_Mars = 4000.0
x_Tonoz_Neptunus = 3000.0
x_White_Kiosk_Atlas = 250.0
x_White_Kiosk_Minerva = 4750.0
